In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 

In [2]:
train = pd.read_csv("modifiedpage.csv", header=None)
train.head()

,0,1,2,3,4,5,6,7,8,9,10
0,48.0,402.0,19296.0,8.375,0.055,0.094,4.13,1058.0,1814.0,256.0,1.0
1,304.0,463.0,140752.0,1.523,0.063,0.121,3.91,8898.0,17081.0,2273.0,1.0
2,306.0,465.0,142290.0,1.520,0.055,0.123,2.69,7861.0,17452.0,2925.0,1.0
3,45.0,79.0,3555.0,1.756,0.087,0.195,4.81,308.0,693.0,64.0,1.0
4,311.0,463.0,143993.0,1.489,0.088,0.160,3.93,12631.0,23092.0,3212.0,1.0


In [3]:
features = train.columns[0:10]
X = train[features]
y = train[10]
X.head()

,0,1,2,3,4,5,6,7,8,9
0,48.0,402.0,19296.0,8.375,0.055,0.094,4.13,1058.0,1814.0,256.0
1,304.0,463.0,140752.0,1.523,0.063,0.121,3.91,8898.0,17081.0,2273.0
2,306.0,465.0,142290.0,1.520,0.055,0.123,2.69,7861.0,17452.0,2925.0
3,45.0,79.0,3555.0,1.756,0.087,0.195,4.81,308.0,693.0,64.0
4,311.0,463.0,143993.0,1.489,0.088,0.160,3.93,12631.0,23092.0,3212.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=40)


In [5]:
print(X_train.shape,X_test.shape)

(4378, 10) (1095, 10)


In [6]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

In [7]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=90.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,1095):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,1095):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
             pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

# FSVM using Hyperplane

In [8]:
from cvxopt import matrix
class HYP_SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        print(self.m.shape)
        
 ##############################################################################


    def fit(self, X_train,X_test, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [9]:

if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        clf = HYP_SVM(C=100.0)
        typ=2
        clf.m_func(X_train,X_test,y_train)
        clf.fit(X_train,X_test, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    hyp_svm()      

(4378, 4378)
     pcost       dcost       gap    pres   dres
 0:  1.0582e+05 -3.9397e+06  7e+06  8e-02  1e-13
 1:  1.1514e+05 -1.2167e+06  2e+06  2e-02  1e-13
 2:  8.1293e+04 -5.4305e+05  7e+05  5e-03  1e-13
 3:  6.1923e+04 -4.0999e+05  5e+05  3e-03  1e-13
 4:  3.5823e+04 -2.4480e+05  3e+05  1e-03  1e-13
 5:  1.3486e+04 -1.0724e+05  1e+05  5e-04  1e-13
 6:  7.4089e+03 -7.9322e+04  9e+04  3e-04  1e-13
 7:  7.9504e+02 -4.5489e+04  5e+04  1e-04  2e-13
 8: -1.0602e+03 -3.6858e+04  4e+04  8e-05  2e-13
 9: -3.8398e+03 -2.3249e+04  2e+04  4e-05  2e-13
10: -4.4742e+03 -1.9919e+04  2e+04  3e-05  2e-13
11: -5.0104e+03 -1.7253e+04  1e+04  1e-05  2e-13
12: -5.1005e+03 -1.6516e+04  1e+04  1e-05  2e-13
13: -5.8502e+03 -1.2499e+04  7e+03  3e-07  2e-13
14: -6.1927e+03 -1.1687e+04  5e+03  2e-07  2e-13
15: -6.5780e+03 -9.8601e+03  3e+03  4e-09  2e-13
16: -6.7152e+03 -9.4407e+03  3e+03  2e-09  2e-13
17: -6.9337e+03 -8.6871e+03  2e+03  1e-09  2e-13
18: -7.0752e+03 -8.1239e+03  1e+03  1e-10  2e-13
19: -7.1

In [10]:
clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

Overall RBF KERNEL SVM accuracy:  0.9799086757990868


# Normal SVM using CVXOPT

In [11]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [12]:

if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    normal_svm()    

(4378, 4378)
     pcost       dcost       gap    pres   dres
 0:  1.0582e+05 -3.9397e+06  7e+06  8e-02  1e-13
 1:  1.1514e+05 -1.2167e+06  2e+06  2e-02  1e-13
 2:  8.1293e+04 -5.4305e+05  7e+05  5e-03  1e-13
 3:  6.1923e+04 -4.0999e+05  5e+05  3e-03  1e-13
 4:  3.5823e+04 -2.4480e+05  3e+05  1e-03  1e-13
 5:  1.3486e+04 -1.0724e+05  1e+05  5e-04  1e-13
 6:  7.4089e+03 -7.9322e+04  9e+04  3e-04  1e-13
 7:  7.9504e+02 -4.5489e+04  5e+04  1e-04  2e-13
 8: -1.0602e+03 -3.6858e+04  4e+04  8e-05  2e-13
 9: -3.8398e+03 -2.3249e+04  2e+04  4e-05  2e-13
10: -4.4742e+03 -1.9919e+04  2e+04  3e-05  2e-13
11: -5.0104e+03 -1.7253e+04  1e+04  1e-05  2e-13
12: -5.1005e+03 -1.6516e+04  1e+04  1e-05  2e-13
13: -5.8502e+03 -1.2499e+04  7e+03  3e-07  2e-13
14: -6.1927e+03 -1.1687e+04  5e+03  2e-07  2e-13
15: -6.5780e+03 -9.8601e+03  3e+03  4e-09  2e-13
16: -6.7152e+03 -9.4407e+03  3e+03  2e-09  2e-13
17: -6.9337e+03 -8.6871e+03  2e+03  1e-09  2e-13
18: -7.0752e+03 -8.1239e+03  1e+03  1e-10  2e-13
19: -7.1